In [5]:
import pandas as pd

X_df = pd.read_feather('./preprocessed_data/x_new.feather')
y = pd.read_csv('./preprocessed_data/y_new.csv')
y = y['score']

In [12]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, test_size=0.2, stratify=y
)
X_train = X_train.dropna()
X_test = X_test.dropna()

import random

sgd = SGDClassifier(
    loss='log_loss',
    warm_start=True,
    n_jobs=-1
)

sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)
accuracy_score(y_test, y_pred)

0.8029197080291971

In [13]:
# # DOESN'T IMPROVE ACCURACY
# grid = {
#     'alpha': [0.0001, 0.001, 0.01, 0.1],
#     'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
#     'eta0': [0.001, 0.01, 0.1],
# }

# new_sgd = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-3, random_state=42, n_jobs=-1)

# from sklearn.model_selection import GridSearchCV

# search = GridSearchCV(new_sgd, grid)
# search.fit(X_train, y_train)

# model = search.best_estimator_
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Ensemble model accuracy after tuning: {accuracy}')

Ensemble model accuracy after tuning: 0.8029197080291971


In [4]:
# # DOESN'T IMPROVE ACCURACY
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import accuracy_score

# sgd = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-3, random_state=42, n_jobs=-1)
# rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
# nb = MultinomialNB()

# ensemble = VotingClassifier(estimators=[
#     ('sgd', sgd),
#     ('rf', rf),
#     ('nb', nb)
# ], voting='hard', n_jobs=-1)

# param_grid = {
#     'sgd__alpha': [0.0001, 0.001, 0.01, 0.1],
#     'sgd__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
#     'sgd__eta0': [0.001, 0.01, 0.1],
#     'rf__n_estimators': [100, 200, 300],
#     'rf__max_depth': [10, 20, 30, None],
#     'rf__max_features': ['sqrt', 'log2'],
#     'rf__min_samples_split': [2, 5, 10],
#     'rf__min_samples_leaf': [1, 2, 4],
#     'nb__alpha': [0.5, 1.0, 1.5, 2.0]
# }

# grid_search = RandomizedSearchCV(estimator=ensemble, param_distributions=param_grid, n_iter250,
#                            cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# best_ensemble = grid_search.best_estimator_

# y_pred = best_ensemble.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Ensemble model accuracy after tuning: {accuracy}')
# print(f'Best parameters: {best_params}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END nb__alpha=0.5, rf__max_depth=None, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=2, rf__n_estimators=100, sgd__alpha=0.001, sgd__eta0=0.1, sgd__learning_rate=optimal; total time=   3.0s
[CV] END nb__alpha=0.5, rf__max_depth=None, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=2, rf__n_estimators=100, sgd__alpha=0.001, sgd__eta0=0.1, sgd__learning_rate=optimal; total time=   3.1s
[CV] END nb__alpha=0.5, rf__max_depth=None, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=2, rf__n_estimators=100, sgd__alpha=0.001, sgd__eta0=0.1, sgd__learning_rate=optimal; total time=   3.3s
[CV] END nb__alpha=0.5, rf__max_depth=None, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=2, rf__n_estimators=100, sgd__alpha=0.001, sgd__eta0=0.1, sgd__learning_rate=optimal; total time=   3.5s
[CV] END nb__alpha=0.5, rf__max_depth=None, rf__max_features=sqrt, 

In [ ]:
# import joblib

# joblib.dump(sgd, 'vasco_model.sav')

['vasco_model.sav']